# JSQuAD

事前学習済みT5モデルを、[JGLUE](https://github.com/yahoojapan/JGLUE)のJSQuADタスク用に転移学習（ファインチューニング）します。

# ライブラリやデータの準備

In [1]:
!nvidia-smi

Tue Jul 12 13:52:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 依存ライブラリのインストール

In [2]:
!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2 torchaudio==0.7.2
!pip install -q transformers==4.20.1 pytorch_lightning==1.2.1 sentencepiece

     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 6.9 MB 28.2 MB/s 
     |████████████████████████████████| 12.8 MB 26.2 MB/s 
     |████████████████████████████████| 7.6 MB 8.0 MB/s 
     |████████████████████████████████| 4.4 MB 4.0 MB/s 
     |████████████████████████████████| 814 kB 33.9 MB/s 
     |████████████████████████████████| 1.2 MB 48.5 MB/s 
     |████████████████████████████████| 6.6 MB 42.1 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 140 kB 71.1 MB/s 
     |████████████████████████████████| 829 kB 46.5 MB/s 
     |████████████████████████████████| 1.1 MB 57.5 MB/s 
     |████████████████████████████████| 271 kB 58.9 MB/s 
     |████████████████████████████████| 144 kB 70.4 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 


## 各種ディレクトリ作成

* data: 学習用データセット格納用
* model: 学習済みモデル格納用


In [3]:
!mkdir -p /content/data /content/model
!rm -rf /content/model/*

In [4]:
# 事前学習済みモデル
PRETRAINED_MODEL_NAME = "sonoisa/t5-base-japanese"

# 転移学習済みモデルを保存する場所
MODEL_DIR = "/content/model"

## JGLUEコーパスのダウンロード

In [5]:
!git clone https://github.com/yahoojapan/JGLUE

Cloning into 'JGLUE'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 66 (delta 13), reused 64 (delta 11), pack-reused 0
Unpacking objects: 100% (66/66), done.


In [6]:
# !wget -O amazon_reviews_multilingual_JP_v1_00.tsv.gz https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz

In [7]:
# !pip install -r /content/JGLUE/preprocess/requirements.txt

In [8]:
# !cd /content/JGLUE/preprocess/marc-ja/scripts; \
#     gzip -dc /content/amazon_reviews_multilingual_JP_v1_00.tsv.gz | \
#     python marc-ja.py \
#          --positive-negative \
#          --output-dir ../../../datasets/marc_ja-v1.0 \
#          --max-char-length 500 \
#          --filter-review-id-list-valid ../data/filter_review_id_list/valid.txt \
#          --label-conv-review-id-list-valid ../data/label_conv_review_id_list/valid.txt

## 文字列の正規化の定義

表記揺れを減らします。今回は[neologdの正規化処理](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja)を一部改変したものを利用します。
処理の詳細はリンク先を参照してください。

In [9]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

## データセットを正規化しT5用に形式を整える。

In [10]:
import json


def normalize_text(text):
    text = text.strip()
    assert "\t" not in text
    assert "\r" not in text
    assert "\n" not in text
    assert len(text) > 0

    text = normalize_neologd(text)
    text = text.lower()
    return text


def make_squad_data(json_data):
    data = []
    for datum in json_data["data"]:
        for paragraph in datum["paragraphs"]:
            context = paragraph["context"]
            context = normalize_text(context).replace("[sep]", "<|n|>")
            for qa in paragraph["qas"]:
                qa_id = qa["id"]

                question = qa["question"]
                question = normalize_text(question)

                answer_text = qa["answers"][0]["text"]
                answer_text = normalize_text(answer_text)
                
                input = f"question: {question} context: {context}"
                target = f"{answer_text}"

                data.append((qa_id, input, target))
    return data

with open("/content/JGLUE/datasets/jsquad-v1.0/train-v1.0.json", "r", encoding="utf-8") as f_in:
    json_data = json.load(f_in)
    train_data = make_squad_data(json_data)

with open("/content/JGLUE/datasets/jsquad-v1.0/valid-v1.0.json", "r", encoding="utf-8") as f_in:
    json_data = json.load(f_in)
    test_data = make_squad_data(json_data)

In [11]:
import json
from tqdm import tqdm
from transformers import T5Tokenizer


# 注意: JSQuADのF1値計算の都合で、トークンの間に半角空白を入れた文字列に変換する。
def decode_to_whitespace_delimited_tokens(tokenizer, ids):
    tokens = [tokenizer.decode([id], skip_special_tokens=True).strip() for id in ids]
    tokens = [token for token in tokens if token != ""]
    return " ".join(tokens).strip()


def normalize_squad_test_data(json_data):
    tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

    for datum in tqdm(json_data["data"]):
        for paragraph in datum["paragraphs"]:
            context = paragraph["context"]
            context = normalize_text(context).replace("[sep]", "<|n|>")
            paragraph["context"] = context

            for qa in paragraph["qas"]:
                question = qa["question"]
                question = normalize_text(question)
                qa["question"] = question

                for answer in qa["answers"]:
                    answer_text = answer["text"]
                    answer_text = normalize_text(answer_text)

                    answer_ids = tokenizer.encode(answer_text)
                    answer_text = decode_to_whitespace_delimited_tokens(tokenizer, answer_ids)

                    answer["text"] = answer_text


with open("/content/JGLUE/datasets/jsquad-v1.0/valid-v1.0.json", "r", encoding="utf-8") as f_in, \
    open(f"/content/data/normalized-valid-v1.0.json", "w", encoding="utf-8") as f_test:

    json_data = json.load(f_in)
    normalize_squad_test_data(json_data)
    json.dump(json_data, f_test, ensure_ascii=False)

Downloading:   0%|          | 0.00/785k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

100%|██████████| 59/59 [00:39<00:00,  1.50it/s]


## データ分割

データセットを95% : 5%の比率でtrain/devに分割します。

* trainデータ: 学習に利用するデータ
* devデータ: 学習中の精度評価等に利用するデータ
* testデータ: 学習結果のモデルの精度評価に利用するデータ

In [12]:
import random
from tqdm import tqdm

random.seed(1234)
random.shuffle(train_data)

data_size = len(train_data)
train_ratio = 0.95

def assert_field(field):
    assert len(field) > 0
    assert "\t" not in field
    assert "\n" not in field
    assert "\r" not in field

def to_line(data):
    qa_id, input, target = data
    qa_id = qa_id.strip()
    input = input.strip()
    target = target.strip()

    assert_field(qa_id)
    assert_field(input)
    assert_field(target)

    return f"{qa_id}\t{input}\t{target}\n"

with open(f"/content/data/train.tsv", "w", encoding="utf-8") as f_train, \
    open(f"/content/data/dev.tsv", "w", encoding="utf-8") as f_dev, \
    open(f"/content/data/test.tsv", "w", encoding="utf-8") as f_test:
    
    for i, data in tqdm(enumerate(train_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        else:
            f_dev.write(line)
    
    for i, data in tqdm(enumerate(test_data)):
        line = to_line(data)
        f_test.write(line)


62859it [00:00, 209642.66it/s]
4442it [00:00, 169743.97it/s]


作成されたデータを確認します。

形式: {QA_ID}\t{question: 質問文 context: コンテキスト}\t{答え}

In [13]:
!head -3 data/test.tsv

a10336p0q0	question: 日本で梅雨がないのは北海道とどこか。 context: 梅雨<|n|>梅雨(つゆ、ばいう)は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の南部から長江流域にかけての沿海部、および台湾など、東アジアの広範囲においてみられる特有の気象現象で、5月から7月にかけて来る曇りや雨の多い期間のこと。雨季の一種である。	小笠原諸島
a10336p0q1	question: 梅雨とは何季の一種か? context: 梅雨<|n|>梅雨(つゆ、ばいう)は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の南部から長江流域にかけての沿海部、および台湾など、東アジアの広範囲においてみられる特有の気象現象で、5月から7月にかけて来る曇りや雨の多い期間のこと。雨季の一種である。	雨季
a10336p0q2	question: 梅雨は、世界的にどのあたりで見られる気象ですか? context: 梅雨<|n|>梅雨(つゆ、ばいう)は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の南部から長江流域にかけての沿海部、および台湾など、東アジアの広範囲においてみられる特有の気象現象で、5月から7月にかけて来る曇りや雨の多い期間のこと。雨季の一種である。	東アジアの広範囲


# 学習に必要なクラス等の定義

学習にはPyTorch/PyTorch-lightning/Transformersを利用します。

In [14]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [15]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir="/content/data",  # データセットのディレクトリ
    model_name_or_path=PRETRAINED_MODEL_NAME,
    tokenizer_name_or_path=PRETRAINED_MODEL_NAME,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)

## TSVデータセットクラス

TSV形式のファイルをデータセットとして読み込みます。  
形式は"{QA_ID}\t{question: 質問文 context: コンテキスト}\t{答え}"です。

In [16]:
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.qa_ids = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        qa_id = self.qa_ids[index]

        return {"source_ids": source_ids, "source_mask": source_mask, 
                "target_ids": target_ids, "target_mask": target_mask,
                "qa_id": qa_id}

    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 3
                assert len(line[0]) > 0
                assert len(line[1]) > 0
                assert len(line[2]) > 0

                qa_id = line[0].strip()
                input = line[1].strip()
                target = line[2].strip()

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [input], max_length=self.input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    [target], max_length=self.target_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)
                self.qa_ids.append(qa_id)


試しにテストデータ（test.tsv）を読み込み、トークナイズ結果をみてみます。

In [18]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

# テストデータセットの読み込み
train_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "test.tsv", 
                           input_max_len=512, target_max_len=16)

テストデータの1レコード目をみてみます。

In [19]:
for data in train_dataset:
    print("A. ID")
    print(data["qa_id"])
    print()
    print("B. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("C. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("D. 出力データの元になる文字列")
    print(tokenizer.decode(data["target_ids"]))
    print()
    print("E. 出力データ（Cの文字列がトークナイズされたトークンID列）")
    print(data["target_ids"])
    break

A. ID
a10336p0q0

B. 入力データの元になる文字列
question: 日本で梅雨がないのは北海道とどこか。 context: 梅雨<|n|>梅雨(つゆ、ばいう)は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の南部から長江流域にかけての沿海部、および台湾など、東アジアの広範囲においてみられる特有の気象現象で、5月から7月にかけて来る曇りや雨の多い期間のこと。雨季の一種である。</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

## 学習処理クラス

[PyTorch-Lightning](https://github.com/PyTorchLightning/pytorch-lightning)を使って学習します。

PyTorch-Lightningとは、機械学習の典型的な処理を簡潔に書くことができるフレームワークです。

In [20]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, is_fast=False)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        """バリデーション完了処理"""
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("ptl/val_loss", avg_loss, prog_bar=True)

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        self.log("test_loss", loss)
        return {"test_loss": loss}

    def test_epoch_end(self, outputs):
        """テスト完了処理"""
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        self.log("ptl/test_loss", avg_loss, prog_bar=True)

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hparams.learning_rate, 
                          eps=self.hparams.adam_epsilon)

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hparams.warmup_steps, 
            num_training_steps=self.t_total
        )

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="train.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="dev.tsv", args=self.hparams)
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
            )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.hparams.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.hparams.eval_batch_size, 
                          num_workers=4)


# 転移学習を実行

In [21]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  384,  # 入力文の最大トークン数
    "max_target_length": 16,  # 出力文の最大トークン数
    "train_batch_size":  12,
    "eval_batch_size":   32,
    "num_train_epochs":  2,

    "learning_rate": 5e-4,
    "weight_decay": 0.0,
    "adam_epsilon": 1e-8,
    "warmup_steps": 50,
    "gradient_accumulation_steps": 8,
    })
args = argparse.Namespace(**args_dict)


# checkpoint_callback = pl.callbacks.ModelCheckpoint(
#     "/content/checkpoints", 
#     monitor="val_loss", mode="min", save_top_k=1
# )

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    # checkpoint_callback=checkpoint_callback,
)

In [22]:
# 転移学習の実行（GPUを利用すれば1エポック10分程度）
model = T5FineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保存
model.tokenizer.save_pretrained(MODEL_DIR)
model.model.save_pretrained(MODEL_DIR)

del model

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Validating: 0it [00:00, ?it/s]

# 学習済みモデルの読み込み

In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

# テストデータに対する予測精度を評価

In [24]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics
import collections

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "test.tsv", 
                          input_max_len=args.max_input_length, 
                          target_max_len=args.max_target_length)

test_loader = DataLoader(test_dataset, batch_size=32, num_workers=4)

trained_model.eval()

predictions = collections.OrderedDict()
outputs = []
targets = []

def untokenize(ids):
    token_texts = [tokenizer.decode([id], skip_special_tokens=True).strip() for id in ids]
    token_texts = [t for t in token_texts if t != ""]
    return token_texts

# 注意: JSQuADのF1値計算の都合で、トークンの間に半角空白を入れた文字列に変換する。
def decode_to_whitespace_delimited_tokens(sequences):
    return [" ".join(untokenize(ids.cpu().tolist())).strip() for ids in sequences]

for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    qa_ids = batch['qa_id']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    outs = trained_model.generate(input_ids=input_ids, 
        attention_mask=input_mask, 
        max_length=args.max_target_length,
        return_dict_in_generate=True,
        output_scores=True)

    dec = decode_to_whitespace_delimited_tokens(outs.sequences)
    target = decode_to_whitespace_delimited_tokens(batch["target_ids"])

    outputs.extend(dec)
    targets.extend(target)

    for qa_id, output in zip(qa_ids, dec):
        predictions[qa_id] = output


  0%|          | 0/139 [00:00<?, ?it/s]

In [25]:
from transformers.data.metrics.squad_metrics import squad_evaluate
from transformers.data.processors.squad import SquadV2Processor

processor = SquadV2Processor()
examples = processor.get_dev_examples("/content/data", filename="normalized-valid-v1.0.json")

100%|██████████| 59/59 [00:21<00:00,  2.77it/s]


## 精度評価

- EM: 文字列が完全一致した割合
- F1: トークンが一致した割合のF1値

In [26]:
results = squad_evaluate(examples, predictions)
print(f"EM: {results['exact']}\nF1: {results['f1']}")

EM: 90.027014858172
F1: 94.5384618736534
